# Lora 实战

## Step1 导入相关包

In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

## Step2 加载数据集

In [2]:
ds = Dataset.load_from_disk("./data/alpaca_data_zh/")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [3]:
ds[:3]

{'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。',
  '4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。',
  '朱利叶斯·凯撒，又称尤利乌斯·恺撒（Julius Caesar）是古罗马的政治家、军事家和作家。他于公元前44年3月15日被刺杀。 \n\n根据历史记载，当时罗马元老院里一些参议员联合起来策划了对恺撒的刺杀行动，因为他们担心恺撒的统治将给罗马共和制带来威胁。在公元前44年3月15日（又称“3月的艾达之日”），恺撒去参加元老院会议时，被一群参议员包围并被攻击致死。据记载，他身中23刀，其中一刀最终致命。'],
 'input': ['', '输入：4/16', ''],
 'instruction': ['保持健康的三个提示。', '解释为什么以下分数等同于1/4', '朱利叶斯·凯撒是如何死亡的？']}

In [4]:
# print(len("以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。"))

## Step3 数据集预处理

In [5]:
tokenizer = AutoTokenizer.from_pretrained("D:/pretrained_model/LLM-Research/Llama-3___2-1B")
tokenizer

PreTrainedTokenizerFast(name_or_path='D:/pretrained_model/LLM-Research/Llama-3___2-1B', vocab_size=128000, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|end_of_text|>'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004: AddedToken("<|finetune_right_pad_id|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128005: AddedToken("<|reserved_s

In [6]:
tokenizer.padding_side = "right"  # 一定要设置padding_side为right，否则batch大于1时可能不收敛

In [7]:
tokenizer.decode(0), tokenizer.decode(2)

('!', '#')

In [8]:
tokenizer.pad_token, tokenizer.eos_token, tokenizer.eos_token_id

(None, '<|end_of_text|>', 128001)

In [9]:
tokenizer.pad_token_id = 128001

In [10]:
def process_func(example):
    MAX_LENGTH = 384    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ", add_special_tokens=False)
    response = tokenizer(example["output"], add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.eos_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.eos_token_id]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [11]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [12]:
print(tokenized_ds[0]["input_ids"])

[35075, 25, 111505, 69978, 113614, 9554, 126524, 46239, 3490, 72803, 25, 220, 88852, 21043, 118551, 113614, 9554, 126524, 46239, 49543, 16, 13, 111505, 69978, 111006, 108726, 1811, 74257, 36827, 102210, 108562, 40265, 9554, 111006, 114253, 116051, 107471, 65782, 5486, 110774, 65782, 58291, 83994, 126503, 3922, 27327, 113096, 42399, 64209, 104473, 36651, 113614, 3922, 50285, 103229, 120044, 107079, 120772, 91495, 19361, 103129, 35304, 111689, 83747, 33014, 30358, 3490, 17, 13, 111020, 229, 120383, 120522, 102456, 1811, 74257, 36827, 102456, 11883, 17039, 118882, 9554, 107139, 105, 108171, 5486, 53610, 28873, 5486, 37087, 104858, 53953, 34208, 121496, 57942, 103, 96412, 33857, 103167, 9554, 111678, 101828, 103706, 102456, 53953, 3922, 111098, 103048, 45736, 117587, 122603, 121496, 57942, 103, 34208, 117041, 119008, 105610, 118551, 113614, 9554, 120522, 102456, 105369, 33565, 107, 3490, 18, 13, 124022, 94, 120379, 105843, 102780, 1811, 113136, 120379, 33764, 17792, 33014, 113614, 57237, 3

In [13]:
# tokenizer("abc " + tokenizer.eos_token)

In [14]:
print(tokenizer.decode(tokenized_ds[0]["input_ids"]))

Human: 保持健康的三个提示。

Assistant: 以下是保持健康的三个提示：

1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。

2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。

3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。<|end_of_text|>


In [15]:
# tokenizer("呀", add_special_tokens=False) # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性

In [16]:
print(tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"]))))

4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。<|end_of_text|>


## Step4 创建模型

In [18]:
import torch
# 多卡情况，可以去掉device_map="auto"，否则会将模型拆开
# load_in_4bit 开启4bit量化
# bnb_4bit_compute_dtype 4bit量化中的计算类型
# bnb_4bit_use_double_quant 是否开启双重量化 
# bnb_4bit_quant_type 4bit量化类型
model = AutoModelForCausalLM.from_pretrained("D:/pretrained_model/LLM-Research/Llama-3___2-1B", low_cpu_mem_usage=True, 
                                             torch_dtype=torch.bfloat16, load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16,
                                             bnb_4bit_quant_type="nf4", bnb_4bit_use_double_quant=True)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [19]:
for name, param in model.named_parameters():
    print(name, param.shape, param.dtype)

model.embed_tokens.weight torch.Size([128256, 2048]) torch.bfloat16
model.layers.0.self_attn.q_proj.weight torch.Size([2097152, 1]) torch.uint8
model.layers.0.self_attn.k_proj.weight torch.Size([524288, 1]) torch.uint8
model.layers.0.self_attn.v_proj.weight torch.Size([524288, 1]) torch.uint8
model.layers.0.self_attn.o_proj.weight torch.Size([2097152, 1]) torch.uint8
model.layers.0.mlp.gate_proj.weight torch.Size([8388608, 1]) torch.uint8
model.layers.0.mlp.up_proj.weight torch.Size([8388608, 1]) torch.uint8
model.layers.0.mlp.down_proj.weight torch.Size([8388608, 1]) torch.uint8
model.layers.0.input_layernorm.weight torch.Size([2048]) torch.bfloat16
model.layers.0.post_attention_layernorm.weight torch.Size([2048]) torch.bfloat16
model.layers.1.self_attn.q_proj.weight torch.Size([2097152, 1]) torch.uint8
model.layers.1.self_attn.k_proj.weight torch.Size([524288, 1]) torch.uint8
model.layers.1.self_attn.v_proj.weight torch.Size([524288, 1]) torch.uint8
model.layers.1.self_attn.o_proj.we

In [20]:
model.config

LlamaConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "D:/pretrained_model/LLM-Research/Llama-3___2-1B",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit":

## Lora

### PEFT Step1 配置文件

In [21]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(task_type=TaskType.CAUSAL_LM,)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules=None, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

### PEFT Step2 创建模型

In [22]:
model = get_peft_model(model, config)

In [23]:
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='D:/pretrained_model/LLM-Research/Llama-3___2-1B', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'q_proj', 'v_proj'}, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [24]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [ ]:
# model = model.half()  # 当整个模型都是半精度时，需要将adam_epsilon调大
# torch.tensor(1e-8).half() 

In [25]:
model.print_trainable_parameters()

trainable params: 851,968 || all params: 1,236,666,368 || trainable%: 0.0689


## Step5 配置训练参数

In [26]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=32,
    logging_steps=10,
    num_train_epochs=1,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit"
)

## Step6 创建训练器

In [27]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds.select(range(6000)),
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

C:\Users\49207\AppData\Local\Temp\ipykernel_7596\2462369116.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## Step7 模型训练

In [ ]:
trainer.train()

  0%|          | 0/93 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


{'loss': 2.2915, 'grad_norm': 0.27071723341941833, 'learning_rate': 4.4623655913978496e-05, 'epoch': 0.11}
{'loss': 2.3173, 'grad_norm': 0.33209022879600525, 'learning_rate': 3.924731182795699e-05, 'epoch': 0.21}


## Step8 模型推理

In [ ]:
model.eval()
ipt = tokenizer("Human: {}\n{}".format("你好", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(model.device)
tokenizer.decode(model.generate(**ipt, max_length=128, do_sample=True, eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True)

In [ ]:
model.merge_and_unload() 